## Dependencies

In [1]:

from databrew import DataBrew
from glue import GlueWrapper
import boto3
import json
import time
import sys
import numpy

sys.path.append("/home/glue_user/project_lf/ETL-TDD")

glue = GlueWrapper('us-east-1')
databrew = DataBrew('us-east-1')

# Run 1
import stage_claim_into_raw
import stage_policyholder_into_raw
import stage_provider_into_raw

# Run 2
import stage_claim_into_access
import stage_policyholder_into_access
import stage_provider_into_access

# Run 3
import stage_location_into_optimised
import stage_procedure_into_optimised

# Run 4
import stage_policyholder_into_optimised
import stage_provider_into_optimised
import stage_date_into_optimised

# Run 5
import stage_claim_into_optimised


# import glue dependencies
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, DataFrame
from etl.paths.components import Bucket




## Initialise Spark

In [4]:

# initialise spark session
sc = SparkContext()
sc.setLogLevel("ERROR")
glueContext = GlueContext(sc)





ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_2944561/986952780.py:2 

## Delete objects in S3

In [ ]:
# delete everything in the raw, access and optimised tiers
import boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket('test-lf-wm')

# ********************* RESET RAW *********************
# for obj in bucket.objects.filter(Prefix='etl/raw'):
    # s3.Object(bucket.name,obj.key).delete()

# **********************RESET ACCESS ******************
# for obj in bucket.objects.filter(Prefix='etl/access'):
    # s3.Object(bucket.name,obj.key).delete()

# ******************** RESET OPTIMISED ****************
for obj in bucket.objects.filter(Prefix='etl/optimised'):
    s3.Object(bucket.name,obj.key).delete()
    


## Run 1

In [13]:
outputs = []

raw_df, path = stage_claim_into_raw.run(spark, Bucket.TEST)
outputs.append( (raw_df, path) )
raw_df, path = stage_policyholder_into_raw.run(spark, Bucket.TEST)
outputs.append( (raw_df, path) )
raw_df, path = stage_provider_into_raw.run(spark, Bucket.TEST)
outputs.append( (raw_df, path) )

getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)


In [17]:

# flatten the outputs with numpy
dfs = numpy.array(outputs).flatten()[::2]
paths = numpy.array(outputs).flatten()[1::2]

In [18]:
for df in dfs:
    df.printSchema()

root
 |-- claim_id: integer (nullable = true)
 |-- policy_holder_id: integer (nullable = true)
 |-- provider_id: integer (nullable = true)
 |-- date_of_service: string (nullable = true)
 |-- procedure: string (nullable = true)
 |-- total_procedure_cost: float (nullable = true)
 |-- medibank_pays: float (nullable = true)
 |-- medicare_pays: float (nullable = true)
 |-- excess: float (nullable = true)
 |-- out_of_pocket: float (nullable = true)

root
 |-- policy_holder_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- email_address: string (nullable = true)
 |-- insurance_plan_details: string (nullable = true)
 |-- coverage_start_date: string (nullable = true)
 |-- coverage_end_date: string (nullable = true)
 |-- policy_standing: string (nullable = true)

ro

In [7]:
example_path = paths[-1]

# Crawl
glue.delete_crawler(example_path)
glue.create_crawler(example_path)
# glue.start_crawler(example_path)

############## Wait for crawler to finish #####################

glue.wait_for_crawler(example_path)

############## After crawlers finish. #####################

for path in paths:
     
    databrew.create_dataset(path)
    databrew.create_profile_job(path)
    databrew.start_job_run(path)

############## Wait for job to finish #####################

databrew.wait_for_job(example_path)

for path in paths:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")
    # databrew.show_data_profile_link(path)


Couldn't create crawler. 618572314333:test-raw already exists


Crawler not running.


Couldn't create dataset. Dataset test-raw-claim already exists.
Couldn't create data profile job. The job test-raw-claim already exists.
Couldn't create dataset. Dataset test-raw-provider already exists.
Couldn't create data profile job. The job test-raw-provider already exists.


Waiting for job to finish.........................
You can view the data profile for s3://test-lf-wm/etl/raw/claim_db/claim/full/202306051005/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-raw-claim&tab=profile-overview
You can view the data profile for s3://test-lf-wm/etl/raw/claim_db/policyholder/full/202306051005/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-raw-policyholder&tab=profile-overview
You can view the data profile for s3://test-lf-wm/etl/raw/claim_db/provider/full/202306051006/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-raw-provider&tab=profile-overview


## Run 2

In [12]:
outputs = []

access_df, path = stage_claim_into_access.run(spark, Bucket.TEST)
outputs.append( (access_df, path) )
access_df, path = stage_policyholder_into_access.run(spark, Bucket.TEST)
outputs.append( (access_df, path) )
access_df, path = stage_provider_into_access.run(spark, Bucket.TEST)
outputs.append( (access_df, path) )

getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)
s3.Bucket(name='test-lf-wm') etl/raw/claim_db/claim/full


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)
s3.Bucket(name='test-lf-wm') etl/raw/claim_db/policyholder/full


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)
s3.Bucket(name='test-lf-wm') etl/raw/claim_db/provider/full


In [19]:

# flatten the outputs with numpy
access_dfs = numpy.array(outputs).flatten()[::2]
access_paths = numpy.array(outputs).flatten()[1::2]

In [21]:
for df in access_dfs:
    df.printSchema()

root
 |-- claim_id: integer (nullable = true)
 |-- policy_holder_id: integer (nullable = true)
 |-- provider_id: integer (nullable = true)
 |-- date_of_service: string (nullable = true)
 |-- procedure: string (nullable = true)
 |-- total_procedure_cost: float (nullable = true)
 |-- medibank_pays: float (nullable = true)
 |-- medicare_pays: float (nullable = true)
 |-- excess: float (nullable = true)
 |-- out_of_pocket: float (nullable = true)

root
 |-- policy_holder_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- email_address: string (nullable = true)
 |-- insurance_plan_details: string (nullable = true)
 |-- coverage_start_date: string (nullable = true)
 |-- coverage_end_date: string (nullable = true)
 |-- policy_standing: string (nullable = true)

ro

In [15]:
example_path = access_paths[-1]

# Crawl
glue.delete_crawler(example_path)
glue.create_crawler(example_path)
glue.start_crawler(example_path)
glue.wait_for_crawler(example_path)

############## After crawlers finish. #####################
for path in access_paths:
     
    response = databrew.create_dataset(path)
    response = databrew.create_profile_job(path)        
    response = databrew.start_job_run(path)

############## Wait for job to finish #####################
databrew.wait_for_job(example_path)

# Print links.
for path in paths:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")
    # databrew.show_data_profile_link(path)




Waiting for crawler to finish...........
Waiting for job to finish...........................
You can view the data profile for s3://test-lf-wm/etl/access/claim_db/claim/full/202306051354/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-access-claim&tab=profile-overview
You can view the data profile for s3://test-lf-wm/etl/access/claim_db/policyholder/full/202306051355/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-access-policyholder&tab=profile-overview
You can view the data profile for s3://test-lf-wm/etl/access/claim_db/provider/full/202306051355/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-access-provider&tab=profile-overview


In [ ]:
############## Use data profile. #####################

databrew.get_dq_results(paths[0])

## Run 3

In [34]:
run3 = []
df, path = stage_location_into_optimised.run(spark,Bucket.TEST)
run3.append( (df, path) )
df, path = stage_procedure_into_optimised.run(spark,Bucket.TEST)
run3.append( (df, path) )

s3.Bucket(name='test-lf-wm') etl/access/claim_db/policyholder/full


s3.Bucket(name='test-lf-wm') etl/access/claim_db/provider/full


s3.Bucket(name='test-lf-wm') etl/access/claim_db/claim/full


In [35]:
run3_dfs = numpy.array(run3).flatten()[0::2]
run3_paths = numpy.array(run3).flatten()[1::2]

run3_paths, run3_dfs

(array(['s3://test-lf-wm/etl/optimised/location_dim/full/202306052030/',
        's3://test-lf-wm/etl/optimised/procedure_dim/full/202306052030/'],
       dtype=object),
 array([DataFrame[location_key: bigint, address: string, street: string, postcode: string, suburb: string, track_hash: bigint, record_start_ts: string, record_end_ts: string, record_active_flag: smallint, record_upd_ts: string, record_insert_ts: string],
        DataFrame[procedure: string, procedure_key: bigint, track_hash: bigint, record_start_ts: string, record_end_ts: string, record_active_flag: smallint, record_upd_ts: string, record_insert_ts: string]],
       dtype=object))

In [37]:
for run in run3_dfs:
    run.printSchema()
    run.show(2, truncate=False)

root
 |-- location_key: long (nullable = false)
 |-- address: string (nullable = true)
 |-- street: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- suburb: string (nullable = true)
 |-- track_hash: long (nullable = true)
 |-- record_start_ts: string (nullable = false)
 |-- record_end_ts: string (nullable = false)
 |-- record_active_flag: short (nullable = false)
 |-- record_upd_ts: string (nullable = false)
 |-- record_insert_ts: string (nullable = false)



+------------+--------------------------------------------------------------+------------------------------+--------+--------------------+----------+-------------------+-------------------+------------------+-----------------------+-----------------------+
|location_key|address                                                       |street                        |postcode|suburb              |track_hash|record_start_ts    |record_end_ts      |record_active_flag|record_upd_ts          |record_insert_ts       |
+------------+--------------------------------------------------------------+------------------------------+--------+--------------------+----------+-------------------+-------------------+------------------+-----------------------+-----------------------+
|0           |Level 2 85 Rhonda Upper, Jimmyfurt, TAS, 2987                 |Level 2 85 Rhonda Upper       | 2987   | Jimmyfurt          |null      |1970-01-01 00:00:00|2999-12-31 00:00:00|1                 |2023-06-05 10:31:23.2

+----------+-------------+----------+-------------------+-------------------+------------------+-----------------------+-----------------------+
|procedure |procedure_key|track_hash|record_start_ts    |record_end_ts      |record_active_flag|record_upd_ts          |record_insert_ts       |
+----------+-------------+----------+-------------------+-------------------+------------------+-----------------------+-----------------------+
|CT Scan   |0            |null      |1970-01-01 00:00:00|2999-12-31 00:00:00|1                 |2023-06-05 10:31:25.852|2023-06-05 10:31:25.852|
|Blood Test|1            |null      |1970-01-01 00:00:00|2999-12-31 00:00:00|1                 |2023-06-05 10:31:25.852|2023-06-05 10:31:25.852|
+----------+-------------+----------+-------------------+-------------------+------------------+-----------------------+-----------------------+
only showing top 2 rows



In [38]:
example_path = run3_paths[-1]
# Crawl
glue.delete_crawler(example_path)
glue.create_crawler(run3_paths)
glue.start_crawler(example_path)
glue.wait_for_crawler(example_path)

############## After crawlers finish. #####################
for path in run3_paths:
     
    databrew.create_dataset(path)
    databrew.create_profile_job(path)        
    databrew.start_job_run(path)

############## Wait for job to finish #####################
databrew.wait_for_job(example_path)
time.sleep(10)

# Print links.
for path in run3_paths:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")
    # databrew.show_data_profile_link(path)




Couldn't delete crawler. Crawler entry with name test-optimised does not exist


Waiting for crawler to finish...........
create_table_name(): optimisedlocation_dim
create_dataset_name(): test-optimised-location-dim


Couldn't create dataset:
                Dataset test-optimised-location-dim already exists.


get_bucket_name(): test-lf-wm
create_dataset_name(): test-optimised-location-dim
create_job_name(): test-optimised-location-dim


Couldn't create data profile job:
                The job test-optimised-location-dim already exists.


create_job_name(): test-optimised-location-dim
create_table_name(): optimisedprocedure_dim
create_dataset_name(): test-optimised-procedure-dim


Couldn't create dataset:
                Dataset test-optimised-procedure-dim already exists.


get_bucket_name(): test-lf-wm
create_dataset_name(): test-optimised-procedure-dim
create_job_name(): test-optimised-procedure-dim


Couldn't create data profile job:
                The job test-optimised-procedure-dim already exists.


create_job_name(): test-optimised-procedure-dim
create_job_name(): test-optimised-procedure-dim
Waiting for job to finish.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-dim
.create_job_name(): test-optimised-procedure-d

## Run 4

In [47]:
# reload stage_date_into_optimised
import importlib
importlib.reload(DataBrew)

ImportError: module DataBrew not in sys.modules

In [39]:
run4 = []

env=Bucket.TEST

df, path = stage_policyholder_into_optimised.run(spark,env)
run4.append( (df, path) )
df, path = stage_provider_into_optimised.run(spark,env)
run4.append( (df, path) )
df, path = stage_date_into_optimised.run(spark,env)
run4.append( (df, path) )

s3.Bucket(name='test-lf-wm') etl/access/claim_db/policyholder/full


s3.Bucket(name='test-lf-wm') etl/optimised/location_dim/full


s3.Bucket(name='test-lf-wm') etl/access/claim_db/provider/full


s3.Bucket(name='test-lf-wm') etl/optimised/location_dim/full


/home/glue_user/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/glue_user/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [40]:
run4_dfs = numpy.array(run4).flatten()[0::2]
run4_paths = numpy.array(run4).flatten()[1::2]

In [31]:
glue.delete_crawler(example_path)

Couldn't delete crawler. Crawler entry with name test-optimised does not exist


In [ ]:
importlib.reload(glue_databrew)

In [46]:

example_path = run4_paths[-1]

# Crawl
glue.delete_crawler(example_path)
glue.create_crawler(run4_paths)
glue.start_crawler(example_path)
glue.wait_for_crawler(example_path)

############## After crawlers finish. #####################
for path in run4_paths:
     
    databrew.create_dataset(path)
    databrew.create_profile_job(path)        
    databrew.start_job_run(path)

############## Wait for job to finish #####################
databrew.wait_for_job(example_path)

# Print links.
for path in run4_paths:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")
    # databrew.show_data_profile_link(path)




Waiting for crawler to finish..........
create_table_name(): optimisedpolicyholder_dim
create_dataset_name(): test-optimised-policyholder-dim
get_bucket_name(): test-lf-wm
create_dataset_name(): test-optimised-policyholder-dim
create_job_name(): test-optimised-policyholder-dim
create_job_name(): test-optimised-policyholder-dim
create_table_name(): optimisedprovider_dim
create_dataset_name(): test-optimised-provider-dim
get_bucket_name(): test-lf-wm
create_dataset_name(): test-optimised-provider-dim
create_job_name(): test-optimised-provider-dim
create_job_name(): test-optimised-provider-dim
create_table_name(): optimiseddate_dim
create_dataset_name(): test-optimised-date-dim


Couldn't create dataset:
                Dataset test-optimised-date-dim already exists.


get_bucket_name(): test-lf-wm
create_dataset_name(): test-optimised-date-dim
create_job_name(): test-optimised-date-dim


Couldn't create data profile job:
                The job test-optimised-date-dim already exists.


create_job_name(): test-optimised-date-dim
create_job_name(): test-optimised-date-dim
Waiting for job to finish.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_job_name(): test-optimised-date-dim
.create_j

## Run 5

In [6]:
spark = glueContext.spark_session
env = Bucket.TEST

run5 = []
df, path = stage_claim_into_optimised.run(spark,env)
run5.append( (df, path) )

kwargs is {'region_name': 'us-east-1'}
s3.Bucket(name='test-lf-wm') etl/access/claim_db/claim/full


s3.Bucket(name='test-lf-wm') etl/optimised/date_dim/full


s3.Bucket(name='test-lf-wm') etl/optimised/policyholder_dim/full


s3.Bucket(name='test-lf-wm') etl/optimised/procedure_dim/full


s3.Bucket(name='test-lf-wm') etl/optimised/provider_dim/full


In [7]:
run5_dfs = numpy.array(run5).flatten()[0::2]
run5_paths = numpy.array(run5).flatten()[1::2]

In [8]:
run5_dfs[0].printSchema()

root
 |-- claim_id: integer (nullable = true)
 |-- provider_key: long (nullable = true)
 |-- policyholder_key: long (nullable = true)
 |-- procedure_key: long (nullable = true)
 |-- date_key: long (nullable = true)
 |-- total_procedure_cost: float (nullable = true)
 |-- medibank_pays: float (nullable = true)
 |-- medicare_pays: float (nullable = true)
 |-- excess: float (nullable = true)
 |-- out_of_pocket: float (nullable = true)



In [9]:

example_path = run5_paths[-1]

# Crawl
glue.delete_crawler(example_path)
glue.create_crawler(run5_paths)
glue.start_crawler(example_path)
glue.wait_for_crawler(example_path)

############## After crawlers finish. #####################
for path in run5_paths:
     
    databrew.create_dataset(path)
    databrew.create_profile_job(path)        
    databrew.start_job_run(path)

############## Wait for job to finish #####################
databrew.wait_for_job(example_path)

# Print links.
for path in run5_paths:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")
    # databrew.show_data_profile_link(path)




Waiting for crawler to finish...........
Waiting for job to finish.........................
You can view the data profile for s3://test-lf-wm/etl/optimised/claim_fact/full/202306052051/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-optimised-claim-fact&tab=profile-overview


## Load Redshift